<a href="https://colab.research.google.com/github/Pmalik24/Automated_podcast_sumarization__and_highlight_generation/blob/main/speech_to_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import transcripts
import requests
import zipfile
import os
import shutil

url = 'https://github.com/Pmalik24/Automated_podcast_sumarization__and_highlight_generation/raw/main/Cleaned.zip'


# delete file if already exists
if os.path.exists('Cleaned'):
  shutil.rmtree('Cleaned')
os.makedirs('Cleaned', exist_ok=True)

# Download the file
response = requests.get(url)
zip_file_path = '/content/Cleaned.zip'

# Save the file
with open(zip_file_path, 'wb') as file:
    file.write(response.content)

# Unzip
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
      zip_ref.extractall()

# delete zip file
os.remove(zip_file_path)

In [ ]:
## Importing the data
import csv
import requests
from io import StringIO

url = 'https://raw.githubusercontent.com/Pmalik24/Automated_podcast_sumarization__and_highlight_generation/main/final_df_with_transcripts.csv'

response = requests.get(url)
response.raise_for_status()

# Convert to a StringIO
csv_content = StringIO(response.text)

# populate the dictionary
data_dict = {}
csv_reader = csv.DictReader(csv_content)
for i, row in enumerate(csv_reader):
    if i >= 544:  # Only read the first 544 rows
        break
    for header, value in row.items():
        if header not in data_dict:
            data_dict[header] = []
        data_dict[header].append(value)

data_dict['index'] = data_dict.pop('')

data_dict['index'] = [int(i) for i in data_dict['index']]
print(data_dict)


{'episode_number': ['2114', '2113', '2111', '2110', '2109', '2108', '2107', '2106', '2105', '2104', '2103', '2102', '2101', '2100', '2099', '2098', '2097', '2096', '2095', '2094', '2093', '2092', '2091', '2090', '2089', '2088', '2087', '2086', '2085', '2084', '2083', '2082', '2081', '2080', '2079', '2078', '1945', '1571', '1569', '1919', '1918', '1917', '1916', '1915', '1914', '1913', '1912', '1911', '1910', '1909', '1908', '1907', '1906', '1905', '1904', '1903', '1902', '1901', '1900', '1899', '1898', '1897', '1896', '1895', '1894', '1893', '1892', '1891', '1890', '1889', '1888', '1887', '1886', '1885', '1884', '1883', '1882', '1881', '1880', '1879', '1878', '1877', '1876', '1875', '1874', '1873', '1872', '1871', '1870', '1869', '1868', '1867', '1866', '1865', '1864', '1863', '1862', '1861', '1860', '1859', '1858', '1857', '1856', '1855', '1854', '1853', '1852', '1851', '1850', '1849', '1848', '1847', '1846', '1845', '1844', '1843', '1842', '1841', '1840', '1839', '1838', '1837', '183

In [ ]:
import glob
import re

import os
import difflib

data_trans_list = []
for path in data_dict['transcription_file']:
  #Make inital path guess
  file_path = '/content/Cleaned/'+ path[:-4]
  match = re.search(r'^(.*?\d{2,})', file_path)
  if match:
    f = match.group(1)
  else: f = file_path

  pattern = f"{f}*.{'txt'}"
  matching_files = glob.glob(pattern)
  if len(matching_files) != 0:
    with open(matching_files[0], 'r') as file:
      data_trans_list.append(file.read())
  else:
    # if all else fails find the closest matching file
    directory, wrong_filename = os.path.split(file_path)
    all_files = os.listdir(file_path[:25])
    closest_file = difflib.get_close_matches(file_path, all_files, n=1)
    if closest_file:
        matching_files = os.path.join(directory, closest_file[0])
        with open(matching_files, 'r') as file:
          data_trans_list.append(file.read())
    else:
      print("the following file could not be found: ", file_path)

# assign list to key in dictionary
data_dict['transcipt'] = data_trans_list

In [ ]:
# break into testing and training data
from random import shuffle, seed
seed(42)

# shuffle data
index_shuffled = data_dict['index'].copy()
shuffle(index_shuffled)

# break into test and train
split_on = int(len(data_dict['index'])*.7)
train_index = index_shuffled[:split_on]
test_index = index_shuffled[split_on:]

# fill test and train dictioanries
train_x = {'index': [],
           'episode_number':[],
           'title': [],
           'download_url': [],
           'publication_date': [],
           'transcription_file': [],
           'transcipt': []}
test_x = {'index': [],
           'episode_number':[],
           'title': [],
           'download_url': [],
           'publication_date': [],
           'transcription_file': [],
           'transcipt': []}

for i in data_dict['index']:
  for key in data_dict.keys():
    if i in train_index:
      train_x[key].append(data_dict[key][i])
    if i in test_index:
      test_x[key].append(data_dict[key][i])

In [ ]:
!pip install pydub

In [ ]:
# use pre trained model transformers
from pydub import AudioSegment
import requests
import tempfile
import librosa
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC, Wav2Vec2Model
import torch
import os
import gc
"""
RUN IN TERMINAL / WINDOWS COMMAND SHELL
pip install spacy
python -m spacy download en_core_web_sm
"""


processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
# device = torch.device("cpu")  # Use "mps" for Apple Silicon
model.to(device)

def convert_mp3_to_wav(mp3_file_path, wav_file_path):
    audio = AudioSegment.from_mp3(mp3_file_path)
    audio.export(wav_file_path, format="wav")

def download_and_preprocess_audio(url):
    response = requests.get(url)
    if response.status_code != 200:
        return None  # Handle error appropriately in your real scenario

    # Save the MP3 audio file temporarily
    with tempfile.NamedTemporaryFile(suffix='.mp3', delete=False) as tmp_mp3_file:
        tmp_mp3_file.write(response.content)
        tmp_mp3_file_path = tmp_mp3_file.name

    # Convert MP3 to WAV
    tmp_wav_path = tmp_mp3_file_path.replace('.mp3', '.wav')
    convert_mp3_to_wav(tmp_mp3_file_path, tmp_wav_path)

    # Load and resample the WAV audio file using librosa
    audio, sample_rate = librosa.load(tmp_wav_path, sr=16000)
    os.remove(tmp_mp3_file_path)  # Delete the temporary MP3 file
    os.remove(tmp_wav_path)  # Delete the temporary WAV file after processing
    return audio

def test_audio_to_text(audio):
    chunk_size = 16000 * 120  # 120 seconds; adjust based on your needs and system capability
    transcriptions = []
    for i in range(0, len(audio), chunk_size):
        chunk = audio[i:i+chunk_size]
        inputs = processor(chunk, sampling_rate=16000, return_tensors="pt", padding=True)
        inputs = inputs.input_values.to(device)
        with torch.no_grad():
            logits = model(inputs).logits
        predicted_ids = torch.argmax(logits, dim=-1)
        transcription = processor.batch_decode(predicted_ids)[0]
        transcriptions.append(transcription.strip())

    # Combine chunk transcriptions, handling potential repetitions at chunk boundaries
    full_transcription = " ".join(transcriptions)
    return full_transcription


# Test with a small part of an audio file or a dummy audio input


def process_single_url(url):
    audio = download_and_preprocess_audio(url)
    if audio is None:
        print("Failed to process audio.")
        return 'FAILED'

    print("Audio downloaded and processed. Converting to text...")
    text = test_audio_to_text(audio)
    return text

# def process_single_file(filepath):
#     print("Loading audio file...")
#     # Load and resample the WAV audio file using librosa
#     audio, sample_rate = librosa.load(filepath, sr=16000)

#     if audio is None:
#         print("Failed to load audio.")
#         return

#     print("Audio loaded. Converting to text...")
#     text = test_audio_to_text(audio)
#     return text


algo_transcript = []
for i in test_x['download_url']:
    text = process_single_url(i)
    algo_transcript.append(text)
    del text  # Delete the text variable
    gc.collect()

In [ ]:
import csv

# update dictionary
test_x['generated_transcripts'] = algo_transcript

# Open a CSV file for writing
with open('tran_generated.csv', 'w', newline='') as file:
    writer = csv.writer(file)

    # Write the header
    writer.writerow(test_x.keys())

    # Write the data
    for i in range(max(test_x['index'])):
        row = [test_x[key][i] if i < len(test_x[key]) else '' for key in test_x]
        writer.writerow(row)